# Make CMAQ WRF-Like

    author: Barron H. Henderson
    date: 2020-07-30
    

# Overview

    Need: Many tools exist for WRFChem that we would like to use with CMAQ.
    Question: How make a CMAQ file WRF-Like
    Answer: This is pretty easy using PNC.


# Using PseudoNetCDF

* importing library
* Used to open, slice, and create new variables

In [1]:
import PseudoNetCDF as pnc
import numpy as np
from collections import OrderedDict

# Set input paths

Example inputs are available from an ftp site.


In [2]:
# Set paths to input and output files
concpath = 'CCTM_CONC_EXAMPLE.nc'
outpath = 'CCTM_CONC_EXAMPLE.WRF.nc'

In [3]:
# Open CONC file
concf = pnc.pncopen(concpath, format='ioapi')

In [4]:
# Get time objects for convenience
times = concf.getTimes()

# Configure WRF Translations

* WRF has a lot of meta-data. Most will just be copied from an example.
* Projection information will be updated.
* Time data will be updated.

In [5]:
map_proj = {6: 2, 2: 1}[concf.GDTYP]
map_proj_char = {6: 'Polar Stereographic', 2: 'Lambert Conformal'}[concf.GDTYP]
cen_lon, cen_lat = concf.ij2ll(concf.NCOLS / 2, concf.NROWS / 2)
moad_cen_lat = {6: concf.P_ALP * 90.0, 2: concf.YCENT}[concf.GDTYP]
pole_lat = {6: concf.P_ALP * 90.0}.get(concf.GDTYP, 90)

**PNC:/work/ROMO/anaconda_envs/basic38/lib/python3.8/site-packages/PseudoNetCDF/pncwarn.py:24:UserWarning:
  Currently not using:straight_vertical_longitude_from_pole -98.0


In [6]:
props = OrderedDict([
    ('TITLE', ' OUTPUT WRF2CMAQ    '),
    ('START_DATE', times[0].strftime('%Y-%m-%d_%H:%M:%S')),
    ('SIMULATION_START_DATE', times[0].strftime('%Y-%m-%d_%H:%M:%S')),
    ('WEST-EAST_GRID_DIMENSION', concf.NCOLS + 1),
    ('SOUTH-NORTH_GRID_DIMENSION', concf.NROWS + 1),
    ('BOTTOM-TOP_GRID_DIMENSION', concf.NLAYS + 1),
    ('DX', concf.XCELL),
    ('DY', concf.YCELL),
    ('SKEBS_ON', 0),
    ('SPEC_BDY_FINAL_MU', 1),
    ('USE_Q_DIABATIC', 0),
    ('GRIDTYPE', 'C'),
    ('DIFF_OPT', 1),
    ('KM_OPT', 4),
    ('DAMP_OPT', 3),
    ('DAMPCOEF', 0.2),
    ('KHDIF', 0.0),
    ('KVDIF', 0.0),
    ('MP_PHYSICS', 10),
    ('RA_LW_PHYSICS', 4),
    ('RA_SW_PHYSICS', 4),
    ('SF_SFCLAY_PHYSICS', 7),
    ('SF_SURFACE_PHYSICS', 7),
    ('BL_PBL_PHYSICS', 7),
    ('CU_PHYSICS', 1),
    ('SF_LAKE_PHYSICS', 0),
    ('SURFACE_INPUT_SOURCE', 1),
    ('SST_UPDATE', 1),
    ('GRID_FDDA', 1),
    ('GFDDA_INTERVAL_M', 180),
    ('GFDDA_END_H', 9048),
    ('GRID_SFDDA', 1),
    ('SGFDDA_INTERVAL_M', 180),
    ('SGFDDA_END_H', 9048),
    ('HYPSOMETRIC_OPT', 2),
    ('USE_THETA_M', 0),
    ('SF_URBAN_PHYSICS', 0),
    ('SHCU_PHYSICS', 0),
    ('MFSHCONV', 0),
    ('FEEDBACK', 1),
    ('SMOOTH_OPTION', 0),
    ('SWRAD_SCAT', 1.0),
    ('W_DAMPING', 1),
    ('DT', 60.0),
    ('RADT', 20.0),
    ('BLDT', 0.0),
    ('CUDT', 0.0),
    ('AER_OPT', 0),
    ('SWINT_OPT', 0),
    ('AER_TYPE', 1),
    ('AER_AOD550_OPT', 1),
    ('AER_ANGEXP_OPT', 1),
    ('AER_SSA_OPT', 1),
    ('AER_ASY_OPT', 1),
    ('AER_AOD550_VAL', 0.12),
    ('AER_ANGEXP_VAL', 1.3),
    ('AER_SSA_VAL', 1e-45),
    ('AER_ASY_VAL', 1e-45),
    ('MOIST_ADV_OPT', 2),
    ('SCALAR_ADV_OPT', 2),
    ('TKE_ADV_OPT', 2),
    ('DIFF_6TH_OPT', 2),
    ('DIFF_6TH_FACTOR', 0.12),
    ('FGDT', 0.0),
    ('GUV', 1e-04),
    ('GT', 1e-04),
    ('GQ', 1e-05),
    ('IF_RAMPING', 0),
    ('DTRAMP_MIN', 60.0),
    ('GUV_SFC', 0.0),
    ('GT_SFC', 0.0),
    ('GQ_SFC', 0.0),
    ('RINBLW', 250.0),
    ('OBS_NUDGE_OPT', 0),
    ('BUCKET_MM', -1.0),
    ('BUCKET_J', -1.0),
    ('PREC_ACC_DT', 60.0),
    ('SF_OCEAN_PHYSICS', 0),
    ('ISFTCFLX', 0),
    ('ISHALLOW', 0),
    ('ISFFLX', 1),
    ('ICLOUD', 1),
    ('ICLOUD_CU', 0),
    ('TRACER_PBLMIX', 1),
    ('SCALAR_PBLMIX', 0),
    ('YSU_TOPDOWN_PBLMIX', 0),
    ('GRAV_SETTLING', 0),
    ('DFI_OPT', 0),
    ('SIMULATION_INITIALIZATION_TYPE', 'REAL-DATA CASE'),
    ('WEST-EAST_PATCH_START_UNSTAG', 1),
    ('WEST-EAST_PATCH_END_UNSTAG',  concf.NCOLS),
    ('WEST-EAST_PATCH_START_STAG', 1),
    ('WEST-EAST_PATCH_END_STAG',  concf.NCOLS + 1),
    ('SOUTH-NORTH_PATCH_START_UNSTAG', 1),
    ('SOUTH-NORTH_PATCH_END_UNSTAG',  concf.NROWS),
    ('SOUTH-NORTH_PATCH_START_STAG', 1),
    ('SOUTH-NORTH_PATCH_END_STAG',  concf.NROWS + 1),
    ('BOTTOM-TOP_PATCH_START_UNSTAG', 1),
    ('BOTTOM-TOP_PATCH_END_UNSTAG', concf.NLAYS),
    ('BOTTOM-TOP_PATCH_START_STAG', 1),
    ('BOTTOM-TOP_PATCH_END_STAG', concf.NLAYS + 1),
    ('GRID_ID', 1),
    ('PARENT_ID', 0),
    ('I_PARENT_START', 0),
    ('J_PARENT_START', 0),
    ('PARENT_GRID_RATIO', 1),
    ('CEN_LAT', cen_lat),
    ('CEN_LON', cen_lon),
    ('TRUELAT1', concf.P_BET),
    ('TRUELAT2', concf.P_ALP),
    ('MOAD_CEN_LAT', moad_cen_lat),
    ('STAND_LON', concf.P_GAM),
    ('POLE_LAT', pole_lat),
    ('POLE_LON', 0.0),
    ('GMT', 0.0),
    ('JULYR', times[0].year),
    ('JULDAY', int(times[0].strftime('%j'))),
    ('MAP_PROJ', map_proj),
    ('MAP_PROJ_CHAR', map_proj_char),
    ('MMINLU', 'MODIFIED_IGBP_MODIS_NOAH'),
    ('NUM_LAND_CAT', 20),
    ('ISWATER', 17),
    ('ISLAKE', -1),
    ('ISICE', 15),
    ('ISURBAN', 13),
    ('ISOILWATER', 14),
    ('history', '')
])

# Create a WRF-style output file

* Define dimensions
* Add properties

In [7]:
outf = pnc.PseudoNetCDFFile()
outf.createDimension('Time', len(concf.dimensions['TSTEP'])).setunlimited(True)
outf.createDimension('south_north', concf.NROWS)
outf.createDimension('west_east', concf.NCOLS)
outf.createDimension('bottom_top', concf.NLAYS)
outf.createDimension('soil_layers_stag', 2)
outf.createDimension('land_cat_stag', props['NUM_LAND_CAT'])
outf.createDimension('south_north_stag', concf.NROWS + 1)
outf.createDimension('west_east_stag', concf.NCOLS + 1)
outf.createDimension('bottom_top_stag', concf.NLAYS + 1)
outf.createDimension('DateStrLen', 19)


<PseudoNetCDF.core._dimensions.PseudoNetCDFDimension object (name = DateStrLen, len = 19) at0x2b362281afa0>

In [8]:
for attrk, attrv in props.items():
    setattr(outf, attrk, attrv)

# Add Time Variable

In [9]:
timev = outf.createVariable('Times', 'c', ('Time', 'DateStrLen'))
times_str = np.array([t.strftime('%F_%H:%M:%S') for t in times], 'S19').view('c').reshape(-1, 19)
timev[:] = times_str

# Add Latitude and Longitude Variables

* Use PseudoNetCDF `ij2ll` function to make lon/lat

In [10]:
I, J = np.meshgrid(np.arange(concf.NCOLS), np.arange(concf.NROWS))
lon, lat = concf.ij2ll(I, J)

In [11]:
xlatv = outf.createVariable('XLAT', 'd', ('Time', 'south_north', 'west_east'))
xlatv.FieldType = 104
xlatv.MemoryOrder = "XY"
xlatv.description = "LATITUDE, SOUTH IS NEGATIVE"
xlatv.units = "degree_north"
xlatv.stagger = ""
xlatv.coordinates = "XLONG XLAT"
xlatv[:] = lat

In [12]:
xlongv = outf.createVariable('XLONG', 'd', ('Time', 'south_north', 'west_east'))
xlongv.FieldType = 104
xlongv.MemoryOrder = "XY"
xlongv.description = "LONGITUDE, SOUTH IS NEGATIVE"
xlongv.units = "degree_north"
xlongv.stagger = ""
xlongv.coordinates = "XLONG XLONG"
xlongv[:] = lon

# Copy Over Variables from CMAQ

* Skip TFLAG, whose data is already in Times
* Copy with renamed dimensions.

In [13]:
for vark, invarv in concf.variables.items():
    if vark == 'TFLAG':
        continue

    varv = outf.copyVariable(invarv, key=vark, dimensions=('Time', 'bottom_top', 'south_north', 'west_east'))
    varv.FieldType = 104
    varv.MemoryOrder = "XYZ"
    varv.description = getattr(invarv, 'var_desc', '').strip()
    varv.units = getattr(invarv, 'units', 'unknown').strip()
    varv.stagger = ""
    varv.coordinates = "XLONG XLAT XTIME"

# Save

In [14]:
outf.save(outpath, verbose=0).close()